In [32]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# caso atualize o Chrome, o programa ja atualiza o WebDriver automaticamente
servico = Service(ChromeDriverManager().install()) 
                  
navegador = webdriver.Chrome(service = servico)

In [34]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
navegador.get("https://www.cbctc.puc-rio.br/Paginas/MeuCB/Provas.aspx")

# Login 
navegador.find_element('xpath', '//*[@id="headerNav"]/div/div/div/ul/li[7]/a').click()
navegador.find_element('xpath', '//*[@id="txtMatricula"]').send_keys("2220354")
navegador.find_element('xpath', '//*[@id="txtSenha"]').send_keys("arte3427")
navegador.find_element(By.ID, 'btnEnviar').click()

# Pegar prova
disciplina = navegador.find_element('xpath', '//*[@id="form1"]/div[3]/div/div/div/div/div[4]/div/div[2]/div[1]/table/tbody/tr/td[2]')
data = navegador.find_element('xpath', '//*[@id="form1"]/div[3]/div/div/div/div/div[4]/div/div[2]/div[1]/table/tbody/tr/td[4]')
horario = navegador.find_element('xpath', '//*[@id="form1"]/div[3]/div/div/div/div/div[4]/div/div[2]/div[1]/table/tbody/tr/td[5]')
print(disciplina.text)
print(data.text)
print(horario.text)

#guardando informacoes de prova
discProva = disciplina.text
dataProva = data.text
horaProva = horario.text
print(dataProva[6:10:])
print(dataProva[3:5])
print(dataProva[0:2])
print(horaProva[0:2])
#Nova aba
aerorio = navegador.find_element('xpath', '//*[@id="form1"]/div[3]/div/div/div/div/div[3]/div/div[2]/div/div[3]/div/a')

# Criar uma instância da classe ActionChains
action = ActionChains(navegador)

# Clicar com o botão de rolagem do mouse no elemento
action.move_to_element(aerorio).click(aerorio).perform()
instagramAeroRio = navegador.find_element('xpath', '//*[@id="docs-internal-guid-5f9434de-7fff-a143-6582-0a294e3ba5f5"]/p/a')
print(instagramAeroRio.text)

#Deslogar do site (sair)
navegador.find_element('xpath', '//*[@id="headerNav"]/div/div/div/ul/li[6]/a[2]').click()

FIS4002
26/03/2024 - TER
17-17:50H
2024
03
26
17
https://instagram.com/aerorio?utm_medium=copy_link


In [35]:
# Pesquisa Google e entra
from selenium.webdriver.common.keys import Keys

navegador.get("https://www.google.com/")
campo = navegador.find_element('xpath', '//*[@id="APjFqb"]')

campo.send_keys("provas ciclo basico")
campo.send_keys(Keys.ENTER)
campo = navegador.find_element('xpath', '//*[@id="rso"]/div[1]/div/div/div[1]/div/div/span/a/h3').click()

In [37]:
from datetime import datetime, timedelta, timezone
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import os
from googleapiclient.errors import HttpError
from datetime import datetime, timezone

SCOPES = ["https://www.googleapis.com/auth/calendar"]

def adicionar_prova(disciplina, inicio, final, service):
    evento = {
        'summary': disciplina,
        'description': "prova",
        'colorId': 6,
        'start': {
            'dateTime': inicio,
            'timeZone': 'Sao_Paulo',  # fuso horário
        },
        'end': {
            'dateTime': final,
            'timeZone': 'Sao_Paulo',  # fuso horário
        },
        'recurrence': [
            "RRULE:FREQ=DAILY;COUNT=3"
        ],
        'attendees': [
            {"email": "dmaraassi@gmail.com"},
            {"email": "marassidiogo@gmail.com"}
        ]
    }
    evento = service.events().insert(calendarId='primary', body=evento).execute()
    print('Evento criado')
    
def main():
    # Autenticação
    creds = None
    if os.path.exists("token.json"):        
        creds = Credentials.from_authorized_user_file("token.json")

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow =  InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)

        with open("token.json", "w") as token:
             token.write(creds.to_json())

    try:        
        service = build("calendar", "v3", credentials=creds)
        
        now = datetime.now().isoformat() + "Z"
        event_result = service.events().list(calendarId='primary', timeMin=now, maxResults=12, singleEvents=True, orderBy="startTime").execute()
        events = event_result.get("items", [])

        formato = "%d/%m/%Y %H:%M:%S"
        data = dataProva[0:11] # 26/03/2024
        inicio = data + ' '+ horaProva[0:2] + ":00:00"
        final = data + ' '+ horaProva[3:5] + ':' + horaProva[6:8] + ":00"
        
        iso_inicio = datetime.strptime(inicio, formato)
        iso_final = datetime.strptime(final, formato)
        print(inicio)
        print(final)
        
        adicionar_prova(discProva, inicio, final, service)
        
        if not events:
            print("No upcoming events found")
            return
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            print(start, event["summary"])
    
        
    except HttpError as error:
        print("Erro ocorreu", error)
             
    


if __name__ == "__main__":
    main()


26/03/2024  17:00:00
26/03/2024  17:50:00
2024-03-27 vvavavavavav
2024-04-04 BBBBBBBB
2024-04-05 ccccccc
2024-04-10 PROVA DO BICALHO
